In [13]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from functools import reduce
import openpyxl
import xlsxwriter

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.util import ngrams
from nltk import ngrams, FreqDist
from nltk.lm import NgramCounter
import string
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

from spacy.lang.en.stop_words import STOP_WORDS

from collections import Counter
import itertools

from sklearn.metrics.pairwise import cosine_similarity

from scipy import spatial

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import matplotlib.pyplot as plt
from yellowbrick.cluster import KElbowVisualizer

from nltk.cluster.kmeans import KMeansClusterer

from scipy import stats
from scipy.stats import pearsonr
import pingouin as pg
import os
import importlib
import utilities
%matplotlib inline

# load 

In [14]:
splist=['cbow_6_ukwac_subtitle','cbow_subtitle','glove_6B','TASA']
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'


### 1: 'cbow_6_ukwac_subtitle'
### 2: 'cbow_subtitle'
### 3: 'glove_6B'
### 4: 'TASA'

## Legend
### min: novelty measure between response and stim, semantic distance with "minimun similarity word"
### disp: dispersion of response set from same subject 
### orig: originality of a response based on freqency of occuring from all subjects
### volume: minimum volume of enclosing ellipsoid of the response set
### fluency: number of responses from a subject

## item code
### b_: box, r_: rope, br_:brick

In [59]:
splist

['cbow_6_ukwac_subtitle', 'cbow_subtitle', 'glove_6B', 'TASA']

In [3]:
def renameLabel(label):
    ret=label
    for i in range(len(splist)):
        ret=ret.replace(splist[i],str(i+1))
    return ret
print([renameLabel(l) for l in splist])
    

['1', '2', '3', '4']


# Study 1

In [49]:
studytag='s1'
file = studytag+'_data_agg.xlsx'
agg=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'
file = studytag+'_yy_agg.csv'
df=pd.read_csv(sourceFolder+file)
agg.columns

Index(['id', 'age', 'gender', 'b_rater1', 'b_rater2', 'b_rater3', 'b_rater4',
       'b_cbowu', 'b_cbows', 'b_cboww', 'b_tasa', 'b_glov', 'r_rater1',
       'r_rater2', 'r_rater3', 'r_rater4', 'r_cbowu', 'r_cbows', 'r_cboww',
       'r_tasa', 'r_glov', 'b_cbowu_a', 'b_cbows_a', 'b_cboww_a', 'b_tasa_a',
       'b_glov_a', 'r_cbowu_a', 'r_cbows_a', 'r_cboww_a', 'r_tasa_a',
       'r_glov_a', 'gf_cfiq', 'gf_letters', 'gf_numbers', 'bicb', 'caq_log',
       'icaa_act', 'icaa_ach', 'ssci_creative_identity',
       'ssci_creative_efficacy', 'n_ffi', 'e_ffi', 'o_ffi', 'a_ffi', 'c_ffi',
       'meta1_r1', 'meta1_r2', 'meta1_r3', 'meta1_r4', 'meta2_r1', 'meta2_r2',
       'meta2_r3', 'meta2_r4'],
      dtype='object')

In [70]:
suffixList=['b_','r_']
sdSpace=['cbowu','cbows','cboww','tasa','glov']
col1=['ssci_creative_efficacy','ssci_creative_identity','icaa_ach','caq_log','gf_letters','gf_numbers']
for suffix in suffixList:
    col2=[suffix+p for p in sdSpace]
    col3=[suffix+'min_'+p for p in splist]+\
            [suffix+'orig_'+p for p in splist]+\
            [suffix+'disp_'+p for p in splist]+[suffix+'volume_'+p for p in splist]+[suffix+'fluency']
    raterCol=[suffix+'rater1',suffix+'rater2',suffix+'rater3',suffix+'rater4']
    allDF=pd.concat([df[col3],agg[col1+col2+raterCol]],axis=1)
    allDF['rater']=allDF[raterCol].mean(axis=1)
    allDF.rename(columns=dict([(c,renameLabel(c)) for c in col3]),inplace=True)
    col3=[renameLabel(c) for c in col3]
    display(np.round(allDF[col3].corr(),2).style.set_table_styles([dict(selector="th",props=[('max-width', '75px')])]))
    display(np.round(allDF.corr().loc[col1+['rater'],col2],2))
    display(np.round(allDF.corr().loc[col1+['rater'],col3],2))

,b_min_1,b_min_2,b_min_3,b_min_4,b_orig_1,b_orig_2,b_orig_3,b_orig_4,b_disp_1,b_disp_2,b_disp_3,b_disp_4,b_volume_1,b_volume_2,b_volume_3,b_volume_4,b_fluency
b_min_1,1,0.78,0.73,0.65,0.21,0.6,-0.16,0.61,-0.08,-0.19,0.01,-0.3,-0.42,-0.44,-0.39,-0.5,-0.33
b_min_2,0.78,1,0.59,0.7,0.09,0.6,-0.26,0.51,-0.16,-0.22,0.01,-0.35,-0.44,-0.44,-0.39,-0.4,-0.35
b_min_3,0.73,0.59,1,0.57,0.18,0.59,-0.02,0.55,-0.08,-0.17,0.06,-0.25,-0.37,-0.41,-0.33,-0.49,-0.28
b_min_4,0.65,0.7,0.57,1,0.03,0.46,-0.32,0.39,-0.23,-0.27,-0.06,-0.36,-0.41,-0.43,-0.38,-0.38,-0.34
b_orig_1,0.21,0.09,0.18,0.03,1,0.3,0.31,0.38,0.39,0.37,0.23,0.31,-0.06,-0.09,-0.03,-0.24,0.13
b_orig_2,0.6,0.6,0.59,0.46,0.3,1,0.07,0.71,-0.12,-0.18,-0.01,-0.27,-0.28,-0.31,-0.25,-0.4,-0.24
b_orig_3,-0.16,-0.26,-0.02,-0.32,0.31,0.07,1,0.05,0.28,0.3,0.07,0.33,0.27,0.25,0.26,0.13,0.3
b_orig_4,0.61,0.51,0.55,0.39,0.38,0.71,0.05,1,0.06,0.02,0.14,-0.07,-0.19,-0.22,-0.14,-0.31,-0.09
b_disp_1,-0.08,-0.16,-0.08,-0.23,0.39,-0.12,0.28,0.06,1,0.97,0.76,0.93,0.39,0.35,0.52,0.14,0.58
b_disp_2,-0.19,-0.22,-0.17,-0.27,0.37,-0.18,0.3,0.02,0.97,1,0.76,0.97,0.53,0.5,0.66,0.32,0.63


,b_cbowu,b_cbows,b_cboww,b_tasa,b_glov
ssci_creative_efficacy,0.06,-0.04,0.06,0.05,0.06
ssci_creative_identity,0.11,0.05,0.10,0.05,0.11
icaa_ach,0.02,-0.07,-0.00,-0.00,-0.05
caq_log,-0.08,-0.09,-0.12,-0.13,-0.14
gf_letters,0.12,0.11,0.07,-0.04,0.05
gf_numbers,0.10,0.11,0.05,-0.03,0.02
rater,0.51,0.44,0.46,0.28,0.28


,b_min_1,b_min_2,b_min_3,b_min_4,b_orig_1,b_orig_2,b_orig_3,b_orig_4,b_disp_1,b_disp_2,b_disp_3,b_disp_4,b_volume_1,b_volume_2,b_volume_3,b_volume_4,b_fluency
ssci_creative_efficacy,0.09,0.01,0.19,0.05,0.07,0.12,0.13,0.14,0.06,0.06,0.08,0.06,0.04,0.04,0.08,0.01,0.08
ssci_creative_identity,0.10,0.05,0.21,0.07,0.11,0.21,0.13,0.12,0.02,0.03,-0.01,0.02,0.05,0.05,0.10,0.01,0.02
icaa_ach,0.05,0.02,0.18,0.06,0.11,0.11,0.09,0.12,0.14,0.15,0.16,0.14,0.11,0.10,0.14,-0.01,0.16
caq_log,-0.04,-0.10,-0.01,-0.09,-0.10,-0.06,0.13,-0.10,0.14,0.14,0.07,0.15,0.13,0.12,0.14,0.06,0.09
gf_letters,0.13,0.10,0.14,-0.02,0.09,0.13,0.08,0.07,0.04,-0.01,0.01,-0.03,-0.13,-0.14,-0.15,-0.19,-0.12
gf_numbers,0.15,0.07,0.18,0.02,0.11,0.09,-0.02,0.15,0.07,0.02,0.04,-0.00,0.04,0.01,0.02,-0.07,0.01
rater,0.60,0.57,0.65,0.48,0.21,0.52,-0.11,0.43,0.03,-0.06,0.13,-0.17,-0.33,-0.35,-0.28,-0.42,-0.25


,r_min_1,r_min_2,r_min_3,r_min_4,r_orig_1,r_orig_2,r_orig_3,r_orig_4,r_disp_1,r_disp_2,r_disp_3,r_disp_4,r_volume_1,r_volume_2,r_volume_3,r_volume_4,r_fluency
r_min_1,1,0.89,0.91,0.8,0.31,0.55,-0.26,0.56,-0,-0.05,0.13,-0.2,-0.34,-0.37,-0.29,-0.31,-0.24
r_min_2,0.89,1,0.85,0.79,0.17,0.46,-0.32,0.46,-0.06,-0.05,0.07,-0.22,-0.4,-0.41,-0.32,-0.3,-0.26
r_min_3,0.91,0.85,1,0.75,0.27,0.53,-0.31,0.54,-0.01,-0.04,0.15,-0.2,-0.36,-0.4,-0.3,-0.35,-0.24
r_min_4,0.8,0.79,0.75,1,0.23,0.38,-0.23,0.4,-0,-0.03,0.07,-0.17,-0.31,-0.36,-0.26,-0.35,-0.21
r_orig_1,0.31,0.17,0.27,0.23,1,0.7,0.22,0.69,0.06,0.05,0.03,0.06,0.13,0.13,0.12,0.11,0.08
r_orig_2,0.55,0.46,0.53,0.38,0.7,1,0.1,0.9,0.12,0.1,0.1,0.02,-0.04,-0.04,-0.02,-0.03,0.01
r_orig_3,-0.26,-0.32,-0.31,-0.23,0.22,0.1,1,0.13,0.25,0.22,0.06,0.34,0.35,0.37,0.31,0.27,0.29
r_orig_4,0.56,0.46,0.54,0.4,0.69,0.9,0.13,1,0.15,0.09,0.17,0.04,-0.02,-0.04,-0.01,-0.07,0.02
r_disp_1,-0,-0.06,-0.01,-0,0.06,0.12,0.25,0.15,1,0.95,0.7,0.9,0.46,0.39,0.56,0.1,0.73
r_disp_2,-0.05,-0.05,-0.04,-0.03,0.05,0.1,0.22,0.09,0.95,1,0.66,0.92,0.49,0.46,0.62,0.21,0.78


,r_cbowu,r_cbows,r_cboww,r_tasa,r_glov
ssci_creative_efficacy,0.26,0.20,0.21,0.27,0.24
ssci_creative_identity,0.32,0.26,0.26,0.32,0.23
icaa_ach,0.14,0.08,0.08,0.21,0.10
caq_log,0.17,0.13,0.13,0.27,0.16
gf_letters,-0.08,0.00,-0.07,0.05,-0.09
gf_numbers,-0.03,-0.06,-0.01,0.01,-0.04
rater,0.51,0.50,0.48,0.47,0.46


,r_min_1,r_min_2,r_min_3,r_min_4,r_orig_1,r_orig_2,r_orig_3,r_orig_4,r_disp_1,r_disp_2,r_disp_3,r_disp_4,r_volume_1,r_volume_2,r_volume_3,r_volume_4,r_fluency
ssci_creative_efficacy,0.27,0.23,0.26,0.21,0.09,0.21,-0.04,0.18,0.15,0.14,0.08,0.13,0.05,0.02,0.03,0.05,0.11
ssci_creative_identity,0.28,0.23,0.24,0.21,0.04,0.22,-0.04,0.20,0.13,0.13,0.10,0.11,-0.00,-0.02,-0.01,-0.01,0.04
icaa_ach,0.20,0.18,0.15,0.20,0.12,0.17,-0.04,0.18,0.11,0.08,0.09,0.08,0.18,0.17,0.20,0.06,0.20
caq_log,0.22,0.19,0.23,0.25,0.12,0.13,0.00,0.18,0.03,0.03,-0.01,-0.03,0.00,-0.02,0.04,-0.08,0.03
gf_letters,0.03,0.06,0.05,0.12,0.07,-0.06,-0.14,-0.12,-0.11,-0.06,-0.07,-0.15,-0.21,-0.19,-0.21,-0.14,-0.16
gf_numbers,0.08,0.08,0.13,0.13,-0.04,-0.09,-0.11,-0.07,-0.02,-0.01,0.03,-0.09,-0.12,-0.14,-0.12,-0.12,-0.09
rater,0.68,0.66,0.66,0.61,0.32,0.46,-0.21,0.46,-0.07,-0.09,-0.01,-0.19,-0.26,-0.28,-0.22,-0.24,-0.20


# Study 2

In [71]:
studytag='s2'
file = studytag+'_data_agg.xlsx'
agg=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'
file = studytag+'_yy_agg.csv'
df=pd.read_csv(sourceFolder+file)
agg.columns

Index(['id', 'age', 'gender', 'bicb', 'caq_log', 'o_aestheticappreciation',
       'o_inquisitiveness', 'o_creativity', 'o_unconventionality', 'b_self',
       'b_r1', 'b_r2', 'b_r3', 'b_cbowu', 'b_cbows', 'b_cboww', 'b_tasa',
       'b_glov', 'r_self', 'r_r1', 'r_r2', 'r_r3', 'r_cbowu', 'r_cbows',
       'r_cboww', 'r_tasa', 'r_glov'],
      dtype='object')

In [73]:
suffixList=['b_','r_']
sdSpace=['cbowu','cbows','cboww','tasa','glov']
col1=['o_creativity','o_unconventionality','caq_log','bicb']
for suffix in suffixList:
    col2=[suffix+p for p in sdSpace]
    col3=[suffix+'min_'+p for p in splist]+\
            [suffix+'orig_'+p for p in splist]+\
            [suffix+'disp_'+p for p in splist]+[suffix+'volume_'+p for p in splist]+[suffix+'fluency']
    raterCol=[suffix+'r1',suffix+'r2',suffix+'r3']
    allDF=pd.concat([df[col3],agg[col1+col2+raterCol+[suffix+'self']]],axis=1)
    allDF['rater']=allDF[raterCol].mean(axis=1)
    allDF.rename(columns=dict([(c,renameLabel(c)) for c in col3]),inplace=True)
    col3=[renameLabel(c) for c in col3]
    display(np.round(allDF[col3].corr(),2).style.set_table_styles([dict(selector="th",props=[('max-width', '75px')])]))
    display(np.round(allDF.corr().loc[col1+['rater']+[suffix+'self'],col2],2))
    display(np.round(allDF.corr().loc[col1+['rater']+[suffix+'self'],col3],2))

,b_min_1,b_min_2,b_min_3,b_min_4,b_orig_1,b_orig_2,b_orig_3,b_orig_4,b_disp_1,b_disp_2,b_disp_3,b_disp_4,b_volume_1,b_volume_2,b_volume_3,b_volume_4,b_fluency
b_min_1,1,0.85,0.83,0.68,0.11,0.47,-0.46,0.56,-0.25,-0.37,-0.14,-0.45,-0.44,-0.44,-0.42,-0.46,-0.47
b_min_2,0.85,1,0.76,0.76,0.02,0.52,-0.53,0.62,-0.35,-0.44,-0.16,-0.53,-0.49,-0.48,-0.47,-0.48,-0.5
b_min_3,0.83,0.76,1,0.62,0.16,0.51,-0.41,0.63,-0.2,-0.3,-0.15,-0.39,-0.36,-0.37,-0.32,-0.41,-0.38
b_min_4,0.68,0.76,0.62,1,0.08,0.55,-0.39,0.55,-0.36,-0.43,-0.11,-0.5,-0.4,-0.41,-0.4,-0.39,-0.45
b_orig_1,0.11,0.02,0.16,0.08,1,0.19,0.14,0.19,0.27,0.28,0.24,0.27,0.15,0.1,0.19,0.07,0.21
b_orig_2,0.47,0.52,0.51,0.55,0.19,1,-0.23,0.61,-0.09,-0.12,0.06,-0.19,-0.18,-0.21,-0.15,-0.32,-0.17
b_orig_3,-0.46,-0.53,-0.41,-0.39,0.14,-0.23,1,-0.37,0.25,0.33,0.16,0.39,0.45,0.45,0.43,0.4,0.4
b_orig_4,0.56,0.62,0.63,0.55,0.19,0.61,-0.37,1,-0.13,-0.18,0.08,-0.25,-0.11,-0.13,-0.05,-0.25,-0.11
b_disp_1,-0.25,-0.35,-0.2,-0.36,0.27,-0.09,0.25,-0.13,1,0.96,0.75,0.92,0.25,0.21,0.42,0.01,0.57
b_disp_2,-0.37,-0.44,-0.3,-0.43,0.28,-0.12,0.33,-0.18,0.96,1,0.73,0.97,0.49,0.46,0.64,0.25,0.67


,b_cbowu,b_cbows,b_cboww,b_tasa,b_glov
o_creativity,0.10,0.15,0.07,-0.08,0.11
o_unconventionality,0.13,0.12,0.05,0.09,0.07
caq_log,0.01,0.02,0.02,-0.11,0.08
bicb,0.02,0.02,0.04,0.05,-0.01
rater,0.49,0.28,0.37,0.03,0.30
b_self,0.02,0.01,0.01,0.01,-0.00


,b_min_1,b_min_2,b_min_3,b_min_4,b_orig_1,b_orig_2,b_orig_3,b_orig_4,b_disp_1,b_disp_2,b_disp_3,b_disp_4,b_volume_1,b_volume_2,b_volume_3,b_volume_4,b_fluency
o_creativity,0.08,0.11,0.19,0.04,0.08,0.03,0.06,-0.01,-0.05,-0.05,-0.15,-0.05,-0.03,-0.07,-0.04,-0.04,-0.05
o_unconventionality,0.12,0.17,0.15,-0.01,0.05,-0.03,-0.04,0.03,0.05,0.01,0.03,0.01,-0.09,-0.11,-0.13,-0.10,-0.07
caq_log,-0.02,-0.03,0.00,0.01,0.09,-0.10,0.02,-0.10,0.05,0.06,0.04,0.07,-0.04,-0.01,-0.02,-0.06,0.05
bicb,0.01,0.05,0.03,-0.07,0.12,-0.06,0.10,-0.04,0.04,0.02,-0.02,0.05,-0.09,-0.08,-0.04,-0.05,-0.02
rater,0.51,0.59,0.64,0.51,0.15,0.27,-0.33,0.44,-0.38,-0.45,-0.25,-0.51,-0.38,-0.40,-0.35,-0.38,-0.37
b_self,0.09,0.06,0.12,0.06,0.09,0.01,0.03,0.15,0.07,0.04,0.09,0.04,0.00,0.03,0.09,0.01,0.06


,r_min_1,r_min_2,r_min_3,r_min_4,r_orig_1,r_orig_2,r_orig_3,r_orig_4,r_disp_1,r_disp_2,r_disp_3,r_disp_4,r_volume_1,r_volume_2,r_volume_3,r_volume_4,r_fluency
r_min_1,1,0.89,0.9,0.79,0.3,0.58,-0.33,0.6,-0.2,-0.28,0.16,-0.35,-0.3,-0.31,-0.25,-0.27,-0.3
r_min_2,0.89,1,0.82,0.78,0.17,0.56,-0.41,0.55,-0.22,-0.27,0.07,-0.37,-0.36,-0.37,-0.29,-0.31,-0.31
r_min_3,0.9,0.82,1,0.73,0.26,0.57,-0.29,0.6,-0.13,-0.21,0.25,-0.29,-0.37,-0.38,-0.3,-0.35,-0.3
r_min_4,0.79,0.78,0.73,1,0.3,0.47,-0.14,0.54,-0.07,-0.15,0.19,-0.21,-0.23,-0.24,-0.21,-0.25,-0.24
r_orig_1,0.3,0.17,0.26,0.3,1,0.34,0.39,0.38,0.04,0.03,0.13,0.03,0.19,0.2,0.14,0.16,0.07
r_orig_2,0.58,0.56,0.57,0.47,0.34,1,-0.17,0.86,-0.08,-0.07,0.12,-0.15,-0.06,-0.07,0.02,-0.05,0.01
r_orig_3,-0.33,-0.41,-0.29,-0.14,0.39,-0.17,1,-0.14,0.37,0.39,0.02,0.41,0.34,0.37,0.31,0.2,0.33
r_orig_4,0.6,0.55,0.6,0.54,0.38,0.86,-0.14,1,-0.05,-0.07,0.14,-0.11,0,-0,0.08,0.01,0.03
r_disp_1,-0.2,-0.22,-0.13,-0.07,0.04,-0.08,0.37,-0.05,1,0.96,0.64,0.93,0.4,0.37,0.6,0.18,0.67
r_disp_2,-0.28,-0.27,-0.21,-0.15,0.03,-0.07,0.39,-0.07,0.96,1,0.6,0.97,0.48,0.46,0.68,0.27,0.73


,r_cbowu,r_cbows,r_cboww,r_tasa,r_glov
o_creativity,0.18,0.08,0.08,0.07,0.15
o_unconventionality,-0.06,-0.01,-0.02,0.11,-0.00
caq_log,0.04,-0.09,-0.02,-0.04,-0.02
bicb,0.16,-0.00,0.03,0.03,0.06
rater,0.50,0.43,0.42,0.42,0.37
r_self,0.18,0.22,0.18,0.29,0.16


,r_min_1,r_min_2,r_min_3,r_min_4,r_orig_1,r_orig_2,r_orig_3,r_orig_4,r_disp_1,r_disp_2,r_disp_3,r_disp_4,r_volume_1,r_volume_2,r_volume_3,r_volume_4,r_fluency
o_creativity,0.14,0.06,0.14,0.16,0.14,-0.00,0.09,0.00,0.15,0.09,0.17,0.07,-0.07,-0.07,0.01,-0.11,0.01
o_unconventionality,0.00,0.01,0.04,0.13,0.09,-0.06,0.11,-0.02,-0.04,-0.09,-0.11,-0.09,-0.09,-0.12,-0.12,-0.16,-0.13
caq_log,0.07,-0.05,0.02,0.09,0.08,0.02,0.14,0.03,0.14,0.11,0.14,0.10,0.06,0.07,0.07,0.01,0.05
bicb,0.05,-0.04,0.06,0.08,0.07,-0.02,0.08,0.03,0.14,0.11,0.10,0.09,0.05,0.07,0.10,0.04,0.06
rater,0.68,0.61,0.66,0.69,0.27,0.32,-0.17,0.43,-0.08,-0.18,0.25,-0.20,-0.27,-0.29,-0.28,-0.24,-0.31
r_self,0.22,0.19,0.22,0.26,0.16,0.16,-0.04,0.20,0.12,0.08,0.16,0.07,0.07,0.08,0.09,0.07,0.14


# Study 3

In [6]:
studytag='s3'
file = studytag+'_data_agg.xlsx'
agg=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'
file = studytag+'_yy_agg.csv'
df=pd.read_csv(sourceFolder+file)
agg.columns

Index(['id', 'age', 'gender', 'br_rater1', 'br_rater2', 'br_rater3', 'br_mean',
       'br_cbowu', 'br_cbows', 'br_cboww', 'br_tasa', 'br_glov', 'gf_series',
       'gf_matrix', 'gf_letters', 'gf_papers', 'gf_ravens', 'gf_numbers',
       'n_ffi', 'e_ffi', 'e_ffi.1', 'a_ffi', 'c_ffi', 'meta1_r1', 'meta1_r2',
       'meta1_r3', 'meta2_r1', 'meta2_r2', 'meta2_r3'],
      dtype='object')

In [9]:
suffixList=['br_']
sdSpace=['cbowu','cbows','cboww','tasa','glov']
col1=['gf_series','gf_matrix', 'gf_letters', 'gf_papers', 'gf_ravens', 'gf_numbers']
for suffix in suffixList:
    col2=[suffix+p for p in sdSpace]
    col3=[suffix+'min_'+p for p in splist]+\
            [suffix+'orig_'+p for p in splist]+\
            [suffix+'disp_'+p for p in splist]+[suffix+'volume_'+p for p in splist]+[suffix+'fluency']
    raterCol=[suffix+'rater1',suffix+'rater2',suffix+'rater3']
    allDF=pd.concat([df[col3],agg[col1+col2+raterCol]],axis=1)
    allDF['rater']=allDF[raterCol].mean(axis=1)
    allDF.rename(columns=dict([(c,renameLabel(c)) for c in col3]),inplace=True)
    col3=[renameLabel(c) for c in col3]
    display(np.round(allDF[col3].corr(),2).style.set_table_styles([dict(selector="th",props=[('max-width', '75px')])]))
    display(np.round(allDF.corr().loc[col1+['rater'],col2],2))
    display(np.round(allDF.corr().loc[col1+['rater'],col3],2))

,br_min_1,br_min_2,br_min_3,br_min_4,br_orig_1,br_orig_2,br_orig_3,br_orig_4,br_disp_1,br_disp_2,br_disp_3,br_disp_4,br_volume_1,br_volume_2,br_volume_3,br_volume_4,br_fluency
br_min_1,1,0.81,0.86,0.78,-0.12,-0.04,-0.17,-0.01,0.33,0.16,0.35,0.05,-0.43,-0.43,-0.32,-0.43,-0.08
br_min_2,0.81,1,0.8,0.75,-0.12,0.11,-0.31,0,0.37,0.29,0.42,0.16,-0.43,-0.41,-0.32,-0.38,0.02
br_min_3,0.86,0.8,1,0.88,-0.15,-0.05,-0.14,-0.13,0.29,0.16,0.26,0.03,-0.43,-0.45,-0.34,-0.46,-0.04
br_min_4,0.78,0.75,0.88,1,-0.15,-0.04,-0.11,-0.12,0.17,0.07,0.22,-0.05,-0.43,-0.42,-0.37,-0.4,-0.12
br_orig_1,-0.12,-0.12,-0.15,-0.15,1,0.25,0.16,0.2,-0.01,0.07,-0.13,0.07,0.21,0.17,0.2,0.07,0.15
br_orig_2,-0.04,0.11,-0.05,-0.04,0.25,1,0.06,0.65,0.18,0.26,0.19,0.25,0.09,0.08,0.19,0.1,0.34
br_orig_3,-0.17,-0.31,-0.14,-0.11,0.16,0.06,1,-0.07,-0.15,-0.12,-0.4,-0.01,0.43,0.41,0.31,0.36,0.09
br_orig_4,-0.01,0,-0.13,-0.12,0.2,0.65,-0.07,1,0.11,0.14,0.19,0.17,0.12,0.09,0.2,0.09,0.29
br_disp_1,0.33,0.37,0.29,0.17,-0.01,0.18,-0.15,0.11,1,0.93,0.76,0.89,-0,-0.07,0.37,-0.26,0.68
br_disp_2,0.16,0.29,0.16,0.07,0.07,0.26,-0.12,0.14,0.93,1,0.71,0.95,0.12,0.07,0.49,-0.11,0.77


,br_cbowu,br_cbows,br_cboww,br_tasa,br_glov
gf_series,0.23,0.21,0.23,0.21,0.18
gf_matrix,0.10,0.03,0.06,0.03,0.04
gf_letters,0.12,0.14,0.16,0.14,0.07
gf_papers,0.11,0.13,0.08,0.12,0.10
gf_ravens,0.11,0.09,0.08,0.08,0.12
gf_numbers,0.05,0.14,0.10,0.14,0.08
rater,0.43,0.41,0.40,0.21,0.29


,br_min_1,br_min_2,br_min_3,br_min_4,br_orig_1,br_orig_2,br_orig_3,br_orig_4,br_disp_1,br_disp_2,br_disp_3,br_disp_4,br_volume_1,br_volume_2,br_volume_3,br_volume_4,br_fluency
gf_series,0.23,0.19,0.20,0.29,-0.18,-0.12,-0.08,-0.10,-0.01,-0.11,0.03,-0.11,-0.28,-0.26,-0.21,-0.27,-0.14
gf_matrix,0.00,-0.00,-0.06,0.03,-0.03,-0.12,0.02,-0.12,-0.10,-0.16,-0.07,-0.15,-0.10,-0.10,-0.13,-0.08,-0.16
gf_letters,0.09,0.07,0.07,0.12,0.03,0.15,0.11,0.02,0.17,0.10,0.08,0.07,-0.15,-0.17,-0.03,-0.17,0.10
gf_papers,0.16,0.13,0.17,0.16,-0.11,0.02,-0.01,-0.01,0.11,0.05,0.10,0.04,-0.17,-0.17,-0.11,-0.22,-0.00
gf_ravens,0.19,0.16,0.18,0.20,-0.06,0.04,0.04,-0.01,0.05,0.00,0.05,-0.05,-0.11,-0.08,-0.05,-0.17,-0.09
gf_numbers,0.11,0.18,0.10,0.15,0.01,0.17,-0.11,0.15,0.15,0.13,0.19,0.10,-0.08,-0.04,-0.08,0.05,0.09
rater,0.58,0.42,0.53,0.48,-0.02,0.24,-0.01,0.27,0.16,0.00,0.17,-0.03,-0.20,-0.21,-0.15,-0.19,-0.01


# Study 5

In [10]:
studytag='s5'
file = studytag+'_data_agg.xlsx'
agg=pd.read_excel(sourceFolder+file,sheet_name=studytag+'_data_agg')
sourceFolder ='/Users/yyu/Box Sync/ORG-SCHOOL-WCAS-PSYCHOLOGY-BEEMAN-LAB/COAT/scoring/SemDisData/rawdata/'
file = studytag+'_yy_agg.csv'
df=pd.read_csv(sourceFolder+file)
agg.columns

Index(['id', 'age', 'gender', 'r1_novelty', 'r1_novelty.1', 'r3_novelty',
       'human_factor', 'cbowu', 'cbows', 'cboww', 'tasa', 'glov',
       'semdis_factor'],
      dtype='object')

In [12]:
suffixList=['']
sdSpace=['cbowu','cbows','cboww','tasa','glov']
col1=['human_factor']
for suffix in suffixList:
    col2=[suffix+p for p in sdSpace]
    col3=[suffix+'min_'+p for p in splist]+\
            [suffix+'orig_'+p for p in splist]+\
            [suffix+'disp_'+p for p in splist]+[suffix+'volume_'+p for p in splist]
    raterCol=[suffix+'r1_novelty',suffix+'r1_novelty.1',suffix+'r3_novelty']
    allDF=pd.concat([df[col3],agg[col1+col2+raterCol+['semdis_factor']]],axis=1)
    allDF['rater']=allDF[raterCol].mean(axis=1)
    allDF.rename(columns=dict([(c,renameLabel(c)) for c in col3]),inplace=True)
    col3=[renameLabel(c) for c in col3]
    display(np.round(allDF[col3].corr(),2).style.set_table_styles([dict(selector="th",props=[('max-width', '75px')])]))
    display(np.round(allDF.corr().loc[col1+['rater'],col2+['semdis_factor']],2))
    display(np.round(allDF.corr().loc[col1+['rater']+['semdis_factor'],col3],2))

,min_1,min_2,min_3,min_4,orig_1,orig_2,orig_3,orig_4,disp_1,disp_2,disp_3,disp_4,volume_1,volume_2,volume_3,volume_4
min_1,1,0.91,0.92,0.85,0.45,0.62,0.75,0.86,0.27,0.29,0.3,0.18,-0,-0.07,-0.03,0
min_2,0.91,1,0.87,0.85,0.33,0.67,0.71,0.82,0.17,0.33,0.25,0.17,-0.01,-0.05,-0.03,-0.01
min_3,0.92,0.87,1,0.81,0.31,0.57,0.69,0.77,0.22,0.29,0.33,0.13,-0.03,-0.08,0.01,0
min_4,0.85,0.85,0.81,1,0.4,0.64,0.71,0.77,0.22,0.31,0.25,0.22,-0.03,-0.03,0.01,0.02
orig_1,0.45,0.33,0.31,0.4,1,0.48,0.66,0.49,0.18,0.1,0.09,0.08,0.01,-0.04,-0.06,-0.01
orig_2,0.62,0.67,0.57,0.64,0.48,1,0.85,0.71,0.08,0.25,0.15,0.13,-0.05,-0.01,-0.04,-0.02
orig_3,0.75,0.71,0.69,0.71,0.66,0.85,1,0.79,0.19,0.25,0.24,0.14,-0.04,-0.05,-0.01,-0
orig_4,0.86,0.82,0.77,0.77,0.49,0.71,0.79,1,0.22,0.27,0.25,0.18,0.02,-0.05,-0.05,0
disp_1,0.27,0.17,0.22,0.22,0.18,0.08,0.19,0.22,1,0.62,0.63,0.52,0.08,0.13,0.02,0.01
disp_2,0.29,0.33,0.29,0.31,0.1,0.25,0.25,0.27,0.62,1,0.57,0.5,0.03,0.16,0.06,0.07


,cbowu,cbows,cboww,tasa,glov,semdis_factor
human_factor,0.85,0.83,0.78,0.78,0.83,0.88
rater,0.82,0.81,0.76,0.78,0.77,0.85


,min_1,min_2,min_3,min_4,orig_1,orig_2,orig_3,orig_4,disp_1,disp_2,disp_3,disp_4,volume_1,volume_2,volume_3,volume_4
human_factor,0.16,0.14,0.04,0.11,-0.01,-0.07,-0.13,0.02,0.08,0.09,0.08,0.15,-0.01,-0.02,0.10,0.12
rater,0.17,0.15,0.04,0.12,-0.02,-0.05,-0.11,0.01,0.05,0.10,0.07,0.17,0.01,-0.02,0.12,0.10
semdis_factor,0.13,0.17,0.05,0.12,0.03,-0.05,-0.11,0.09,0.03,0.00,0.13,0.08,-0.01,-0.02,0.27,0.11
